# Adversarial Simulator for Summarization

In [ ]:
from pathlib import Path
from azure.ai.generative.evaluate import evaluate
import json
from azure.ai.generative.synthetic.simulator import Simulator
from azure.ai.resources.client import AIClient
from azure.identity import DefaultAzureCredential
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
from openai import AsyncAzureOpenAI

sub = ""
rg = ""
project_name = ""
oai_client = AsyncAzureOpenAI(api_key="", azure_endpoint="", api_version="2023-12-01-preview")

## Initialize the simulator, get template and set up parameters

In [ ]:
async_oai_chat_completion_fn = oai_client.chat.completions.create
client = AIClient(
    subscription_id=sub, resource_group_name=rg, project_name=project_name, credential=DefaultAzureCredential()
)

simulator = Simulator.from_fn(fn=async_oai_chat_completion_fn, ai_client=client, model="gpt-4", max_tokens=300)
template_params = [
    {
        "name": "John Doe",
        "chatbot_name": "AI Chatbot",
        "filename": "company_report.txt",
        "file_content": "The company is doing well. The stock price is up 10% this quarter. The company is expanding into new markets. The company is investing in new technology. The company is hiring new employees. The company is launching new products. The company is opening new stores. The company is increasing its market share. The company is increasing its revenue. The company is increasing its profits.",
    },
    {
        "name": "Jane Doe",
        "chatbot_name": "AI Chatbot",
        "filename": "sales_report.txt",
        "file_content": "The sales team is doing well. The sales team is meeting its targets. The sales team is increasing its revenue. The sales team is increasing its market share. The sales team is increasing its profits. The sales team is expanding into new markets. The sales team is launching new products. The sales team is opening new stores. The sales team is hiring new employees. The sales team is investing in new technology.",
    },
]
template = Simulator.get_template("summarization")

## Run the simulator

In [ ]:
outputs = await simulator.simulate_async(
    template,
    parameters=template_params,
    max_conversation_turns=2,
    api_call_delay_sec=10,
    simulation_result_limit=10,
)

## Print the outputs

In [ ]:
for line in outputs:
    print(json.dumps(line, indent=2))

## Store outputs in file for eval

In [ ]:
eval_name = "conv-oai-summarization-eval"
file_name = f"\{eval_name}.jsonl"
jsonl_object = outputs.to_json_lines()
with Path.open(file_name, "w") as f:
    f.write(jsonl_object)

## Run evaluate

In [ ]:
tracking_uri = client.tracking_uri
connection = client.connections.get("Default_AzureOpenAI")
config = AzureOpenAIModelConfiguration.from_connection(connection, model_name="gpt-4", deployment_name="gpt-4")
data_path = Path.cwd() / f"\{eval_name}.jsonl"
output_path = Path.cwd() / "downloaded_artifacts" / "remote"
result = evaluate(
    evaluation_name=eval_name,
    data=data_path,
    task_type="qa",
    metrics_list=["gpt_groundedness", "gpt_relevance", "gpt_coherence", "gpt_fluency", "gpt_similarity"],
    model_config=config,
    data_mapping={"messages": "messages"},
    tracking_uri=tracking_uri,
    output_path=output_path,
)
print(result)
print(result.metrics_summary)  # will print the defect rate for each content harm
print("Studio URL")
print(result.studio_url)